# Hierarchical Indexing


Hierarchical indexing is an important feature of pandas enabling you to have multiple
(two or more) index levels on an axis. Somewhat abstractly, it provides a way for you
to work with higher dimensional data in a lower dimensional form. Let’s start with a
simple example; create a Series with a list of lists or arrays as the index:

In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

In [2]:
data = Series(np.random.randn(10)
            ,index=[['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'd', 'd']
            ,[1, 2, 3, 1, 2, 3, 1, 2, 2, 3]])

In [3]:
data

a  1    2.293825
   2    0.703377
   3    0.586082
b  1    0.092303
   2   -0.457527
   3   -0.930360
c  1   -1.034395
   2    0.026512
d  2    1.625262
   3   -0.752239
dtype: float64

What you’re seeing is a prettified view of a Series with a MultiIndex as its index. The
“gaps” in the index display mean “use the label directly above”:

In [4]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 1, 2, 0, 1, 1, 2]])

With a hierarchically-indexed object, so-called partial indexing is possible, enabling
you to concisely select subsets of the data:

In [5]:
data['b']

1    0.092303
2   -0.457527
3   -0.930360
dtype: float64

In [6]:
data['b':'c']

b  1    0.092303
   2   -0.457527
   3   -0.930360
c  1   -1.034395
   2    0.026512
dtype: float64

In [7]:
data.ix[['b', 'd']]

b  1    0.092303
   2   -0.457527
   3   -0.930360
d  2    1.625262
   3   -0.752239
dtype: float64

Selection is even possible in some cases from an “inner” level:

In [8]:
data[:, 2]

a    0.703377
b   -0.457527
c    0.026512
d    1.625262
dtype: float64

Hierarchical indexing plays a critical role in reshaping data and group-based operations
like forming a pivot table. For example, this data could be rearranged into a DataFrame
using its unstack method:

In [9]:
data.unstack()

,1,2,3
a,2.293825,0.703377,0.586082
b,0.092303,-0.457527,-0.930360
c,-1.034395,0.026512,NaN
d,NaN,1.625262,-0.752239


In [10]:
data.unstack().stack()

a  1    2.293825
   2    0.703377
   3    0.586082
b  1    0.092303
   2   -0.457527
   3   -0.930360
c  1   -1.034395
   2    0.026512
d  2    1.625262
   3   -0.752239
dtype: float64

stack and unstack will be explored in more detail in Chapter 7.
With a DataFrame, either axis can have a hierarchical index:

In [15]:
frame = DataFrame(np.arange(12).reshape(4, 3)
                  ,index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
        columns=[['Ohio', 'Ohio', 'Colorado'],['Green', 'Red', 'Green']])

In [16]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

The hierarchical levels can have names (as strings or any Python objects). If so, these
will show up in the console output (don’t confuse the index names with the axis labels!):

In [18]:
frame.index.names = ['key1', 'key2']

In [32]:
frame.columns.names = ['state', 'color']

In [33]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

With partial column indexing you can similarly select groups of columns:

In [34]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

A MultiIndex can be created by itself and then reused; the columns in the above Data-
Frame with level names could be created like this:m

In [35]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
names=['state', 'color'])

MultiIndex(levels=[['Colorado', 'Ohio'], ['Green', 'Red']],
           labels=[[1, 1, 0], [0, 1, 0]],
           names=['state', 'color'])

## Reordering and Sorting Levels


At times you will need to rearrange the order of the levels on an axis or sort the data
by the values in one specific level. The swaplevel takes two level numbers or names and
returns a new object with the levels interchanged (but the data is otherwise unaltered):

In [36]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

sortlevel, on the other hand, sorts the data (stably) using only the values in a single
level. When swapping levels, it’s not uncommon to also use sortlevel so that the result
is lexicographically sorted:

In [37]:
frame.sortlevel(1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [38]:
frame.swaplevel(0, 1).sortlevel(0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

Data selection performance is much better on hierarchically indexed
objects if the index is lexicographically sorted starting with the outermost
level, that is, the result of calling sortlevel(0) or sort_index().

## Summary Statistics by Level


Many descriptive and summary statistics on DataFrame and Series have a level option
in which you can specify the level you want to sum by on a particular axis. Consider
the above DataFrame; we can sum by level on either the rows or columns like so:

In [39]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [40]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

Under the hood, this utilizes pandas’s groupby machinery which will be discussed in
more detail later in the book.


## Using a DataFrame’s Columns

It’s not unusual to want to use one or more columns from a DataFrame as the row
index; alternatively, you may wish to move the row index into the DataFrame’s columns.
Here’s an example DataFrame:

In [42]:
frame = DataFrame({'a': range(7), 'b': range(7, 0, -1),
        'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
        'd': [0, 1, 2, 0, 1, 2, 3]})

In [43]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


DataFrame’s set_index function will create a new DataFrame using one or more of its
columns as the index:

In [44]:
frame2 = frame.set_index(['c', 'd'])

In [45]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

By default the columns are removed from the DataFrame, though you can leave them in:

In [46]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

reset_index, on the other hand, does the opposite of set_index; the hierarchical index
levels are are moved into the columns:

In [47]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1
